In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from math import sqrt
import cv2
import imutils
import random
from nltk import flatten

In [ ]:
def before_layers(n, x_i, y_i, x_i_1, y_i_1):
    layer_1 = []
    for b1 in range(len(x_i_1)):
        layer_1.append(f"{x_i_1[b1]} {y_i_1[b1]}")

    index = []
    for p in range(len(x_i)):
        current_x = x_i[p]
        current_y = y_i[p]
        i_1 = [list(map(float, x.split())) for x in layer_1]

        distance = []
        for c in i_1:
            distance.append(sqrt(np.subtract(c[0], current_x) ** 2 + np.subtract(c[1], current_y) ** 2))

        argsort_dis = np.argsort(distance)
        index.append(argsort_dis[n])


    return index

In [ ]:
def neighborhood_same_layer(n,x_i, y_i):
    xy = []
    for idx in range(len(x_i)):
        xy.append(f"{x_i[idx]} {y_i[idx]}")
    index = []
    for p in range(len(x_i)):
        current_x = x_i[p]
        current_y = y_i[p]
        xy_cor = [list(map(float, cor.split())) for cor in xy]

        distance = []
        for c in xy_cor:
            distance.append(sqrt(np.subtract(c[0] , current_x)**2 + np.subtract(c[1] , current_y)**2))

        argsort_dis = np.argsort(distance)
        index.append(argsort_dis[n])
    # print(len(index))
    return index

In [ ]:
def get_neighborhood(data, features, depth_layer, state, num_neighbor):
    dictionary = {}

    def creat_neighbor_features():
      feature_neighborhood = []
      if state == '3D':
        for d in range(1, depth_layer+1):
          for n in range(1,num_neighbor+1):
            for name in features:
              feature_neighborhood.append(name+f'-{d}|N{n}')
      elif state == '2D':
        for n in range(1,num_neighbor+1):
            for name in features:
              feature_neighborhood.append(name+f'|N{n}')

      return feature_neighborhood

    feature_in_layer = []
    feature_in_bottom_layer = []
    for name in features:
      globals()[name] = np.array(data[name])
      feature_in_layer.append(name+'_layer')
      feature_in_bottom_layer.append(name+'_bottom')


    # Create dictionary keys  
    feature_neighborhood = creat_neighbor_features()
    dic_keys = list(features[:-1]) + feature_neighborhood + [features[-1]]
    print('dic keys:',dic_keys)
    for f in dic_keys:
      dictionary[f] = []

    # Get data points in each layer
    value_z = np.unique(pos_z)
    len_layer = len(value_z)
    
    for i in range(depth_layer, len_layer):
        # All feature extracted from i_th layer
        # ...i...
        for name in feature_in_layer: 
          globals()[name] = globals()[name[:-6]][pos_z == value_z[i]]
          dictionary[name[:-6]].append(globals()[name].tolist())
        # ...i...

        if state == '3D':
          # All feature extracted from i-n_th layer
          # ...i-n...
          for depth in range(1, depth_layer+1):
            for neighbor in range(num_neighbor):
              for name in feature_in_bottom_layer: 
                globals()[name] = globals()[name[:-7]][pos_z == value_z[i-depth]]
              
              bottom_index = before_layers(neighbor, pos_x_layer, pos_y_layer, pos_x_bottom, pos_y_bottom)

              for name in feature_in_bottom_layer: 
                globals()[name] = globals()[name][bottom_index]
                dictionary[name[:-7]+f'-{depth}|N{neighbor+1}'].append(globals()[name].tolist())

        elif state == '2D':
          # print(len(pos_x_layer))
          # All feature extracted from neighbors of i_th layer
          for neighbor in range(1, num_neighbor+1):
            neighbor_index = neighborhood_same_layer(neighbor, pos_x_layer, pos_y_layer)

            for name in feature_in_layer:
              dictionary[name[:-6]+f'|N{neighbor}'].append(globals()[name][neighbor_index].tolist())
          

              
    

    return dic_keys, dictionary


In [ ]:
def preprocess_each_sample(flag, features , depth_layer, state, num_neighbor):
    if flag == 'Sample 4':
        path_img = '/content/drive/MyDrive/DED Codes/New images/Sample 4'
        data = pd.read_csv('/content/drive/MyDrive/DED Codes/Porosity prediction in DED/DataPoints On Object/DataPoints Sample 4.csv')
        angle = 10

    elif flag == 'Sample 5':
        path_img = '/content/drive/MyDrive/DED Codes/New images/Sample 5'
        data = pd.read_csv('/content/drive/MyDrive/DED Codes/Porosity prediction in DED/DataPoints On Object/DataPoints Sample 5.csv')
        angle = 100

    elif flag == 'Sample 6':
        path_img = '/content/drive/MyDrive/DED Codes/New images/Sample 6'
        data = pd.read_csv('/content/drive/MyDrive/DED Codes/Porosity prediction in DED/DataPoints On Object/DataPoints Sample 6.csv')
        angle = -85

    elif flag == 'Sample 8':
        path_img = '/content/drive/MyDrive/DED Codes/New images/Sample 8'
        data = pd.read_csv('/content/drive/MyDrive/DED Codes/Porosity prediction in DED/DataPoints On Object/DataPoints Sample 8.csv')
        angle = 105

    images = []
    for filename in os.listdir(path_img):
        # print(filename)
        img = cv2.imread(os.path.join(path_img, filename), cv2.IMREAD_GRAYSCALE)
        rot = imutils.rotate_bound(img, angle=angle)
        images.append(rot)

    keys, df = get_neighborhood(data, features, depth_layer,state, num_neighbor)

    return keys, df
    



In [ ]:
def preprocess_each_class(cls, features , depth_layer, state, num_neighbor):
    if cls == 'Class 1':
       sample4 = pd.read_csv('/content/drive/MyDrive/DED Codes/Porosity prediction in DED/DataPoints On Object/DataPoints Sample 4.csv')
       sample5 = pd.read_csv('/content/drive/MyDrive/DED Codes/Porosity prediction in DED/DataPoints On Object/DataPoints Sample 5.csv')
       data = pd.concat([sample4, sample5])

    elif cls == 'Class 2':
       sample6 = pd.read_csv('/content/drive/MyDrive/DED Codes/Porosity prediction in DED/DataPoints On Object/DataPoints Sample 6.csv')
       sample8 = pd.read_csv('/content/drive/MyDrive/DED Codes/Porosity prediction in DED/DataPoints On Object/DataPoints Sample 8.csv')
       data = pd.concat([sample6, sample8])

    keys, df = get_neighborhood(data, features, depth_layer,state, num_neighbor)

    return keys, df


In [ ]:
def preprocess_all_samples(features , depth_layer, state, num_neighbor):
    sample4 = pd.read_csv('/content/drive/MyDrive/DED Codes/Porosity prediction in DED/DataPoints On Object/DataPoints Sample 4.csv')
    sample5 = pd.read_csv('/content/drive/MyDrive/DED Codes/Porosity prediction in DED/DataPoints On Object/DataPoints Sample 5.csv')
    sample6 = pd.read_csv('/content/drive/MyDrive/DED Codes/Porosity prediction in DED/DataPoints On Object/DataPoints Sample 6.csv')
    sample8 = pd.read_csv('/content/drive/MyDrive/DED Codes/Porosity prediction in DED/DataPoints On Object/DataPoints Sample 8.csv')
    data = pd.concat([sample4, sample5, sample6, sample8])

    keys, df = get_neighborhood(data, features, depth_layer,state, num_neighbor)
    return keys, df

**Get selected features**

---



In [ ]:
# 10
selected_features = ['pos_x', 'pos_y', 'pos_z', 'dist', 'sat', 'LongPeak', 'ShortPeak', 'Time', 'XPeak', 'TGrad','Label']

# 6
# selected_features = ['pos_x', 'pos_y', 'pos_z', 'ShortPeak', 'Time', 'XPeak','Label']
# 8
# selected_features = ['pos_x', 'pos_y', 'pos_z', 'dist', 'LongPeak', 'Time', 'XPeak','TGrad','Label']
# 12
# selected_features = ['pos_x', 'pos_y', 'pos_z', 'dist', 'LongPeak', 'ShortPeak', 'Time','TGrad', 'RGrad', 'velo', 'L', 'W','Label']    
# 15   
# selected_features = ['pos_x', 'pos_y', 'pos_z', 'dist', 'peak_temp', 'sat', 'Time', 'YPeak','TGrad', 'BGrad', 'LGrad', 'RGrad', 'velo', 'L', 'W','Label']

# Sera features
# selected_features = ['pos_x', 'pos_y', 'pos_z', 'peak_temp', 'avg_temp', 'LongPeak', 'ShortPeak', 'Time', 'velo','Label']  
# ALL features
# data = pd.read_csv('/content/drive/MyDrive/DED Codes/Porosity prediction in DED/DataPoints On Object/DataPoints Sample 4.csv')
# selected_features = list(data.head())

# selected_features = ['pos_x', 'pos_y', 'pos_z', 'peak_temp', 'avg_temp', 'velo','Label'] 

# count_f = 'Sera'
# print(count_f)

In [ ]:
count_f = len(selected_features)-1
print(count_f)


10


**Scenario 1: 2 layer before, 1 nearest neighbor**

---



In [ ]:

depth_layer = 2
num_neighbor = 1
state = '3D'
scen = 'Scenario 1'

# Each Sample
flag = ['Sample 4','Sample 5','Sample 6','Sample 8']
for f in flag:
    print(f)
    dic_keys, dictionary = preprocess_each_sample(f, selected_features, depth_layer, state, num_neighbor)
    for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
    df = pd.DataFrame.from_dict(dictionary) 
    df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {f}.csv', index = False, header=True)

# Each Class
cls = ['Class 1','Class 2']
for c in cls:
  dic_keys, dictionary = preprocess_each_class(c, selected_features, depth_layer, state, num_neighbor)
  for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
  df = pd.DataFrame.from_dict(dictionary) 
  df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {c}.csv', index = False, header=True)

# All Samples
dic_keys, dictionary = preprocess_all_samples(selected_features, depth_layer, state, num_neighbor)
for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
df = pd.DataFrame.from_dict(dictionary) 
df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme ALL samples.csv', index = False, header=True)

**Scenario 2: 3 layer before, 1 nearest neighbor**

---



In [ ]:
depth_layer = 3
num_neighbor = 1
state = '3D'
scen = 'Scenario 2'

# Each Sample
flag = ['Sample 4','Sample 5','Sample 6','Sample 8']
for f in flag:
    print(f)
    dic_keys, dictionary = preprocess_each_sample(f, selected_features, depth_layer, state, num_neighbor)
    for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
    df = pd.DataFrame.from_dict(dictionary) 
    df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {f}.csv', index = False, header=True)

# Each Class
cls = ['Class 1','Class 2']
for c in cls:
  dic_keys, dictionary = preprocess_each_class(c, selected_features, depth_layer, state, num_neighbor)
  for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
  df = pd.DataFrame.from_dict(dictionary) 
  df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {c}.csv', index = False, header=True)

# All Samples
dic_keys, dictionary = preprocess_all_samples(selected_features, depth_layer, state, num_neighbor)
for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
df = pd.DataFrame.from_dict(dictionary) 
df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme ALL Samples.csv', index = False, header=True)

**Scenario 3: 4 layer before, 1 nearest neighbor**

---



In [ ]:
depth_layer = 4
num_neighbor = 1
state = '3D'
scen = 'Scenario 3'

# Each Sample
flag = ['Sample 4','Sample 5','Sample 6','Sample 8']
for f in flag:
    print(f)
    dic_keys, dictionary = preprocess_each_sample(f, selected_features, depth_layer, state, num_neighbor)
    for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
    df = pd.DataFrame.from_dict(dictionary) 
    df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {f}.csv', index = False, header=True)

# Each Class
cls = ['Class 1','Class 2']
for c in cls:
  dic_keys, dictionary = preprocess_each_class(c, selected_features, depth_layer, state, num_neighbor)
  for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
  df = pd.DataFrame.from_dict(dictionary) 
  df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {c}.csv', index = False, header=True)

# All Samples
dic_keys, dictionary = preprocess_all_samples(selected_features, depth_layer, state, num_neighbor)
for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
df = pd.DataFrame.from_dict(dictionary) 
df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme ALL Samples.csv', index = False, header=True)

**Scenario 4: 5 layer before, 1 nearest neighbor**

---



In [ ]:
depth_layer = 5
num_neighbor = 1
state = '3D'
scen = 'Scenario 4'

# Each Sample
flag = ['Sample 4','Sample 5','Sample 6','Sample 8']
for f in flag:
    print(f)
    dic_keys, dictionary = preprocess_each_sample(f, selected_features, depth_layer, state, num_neighbor)
    for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
    df = pd.DataFrame.from_dict(dictionary) 
    df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {f}.csv', index = False, header=True)

# Each Class
cls = ['Class 1','Class 2']
for c in cls:
  dic_keys, dictionary = preprocess_each_class(c, selected_features, depth_layer, state, num_neighbor)
  for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
  df = pd.DataFrame.from_dict(dictionary) 
  df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {c}.csv', index = False, header=True)

# All Samples
dic_keys, dictionary = preprocess_all_samples(selected_features, depth_layer, state, num_neighbor)
for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
df = pd.DataFrame.from_dict(dictionary) 
df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme ALL Samples.csv', index = False, header=True)

**Scenario 5: 2 layer before, 2 nearest neighbor**

---



In [ ]:
depth_layer = 2
num_neighbor = 2
state = '3D'
scen = 'Scenario 5'

# Each Sample
flag = ['Sample 4','Sample 5','Sample 6','Sample 8']
for f in flag:
    print(f)
    dic_keys, dictionary = preprocess_each_sample(f, selected_features, depth_layer, state, num_neighbor)
    for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
    df = pd.DataFrame.from_dict(dictionary) 
    df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {f}.csv', index = False, header=True)


# Each Class
cls = ['Class 1','Class 2']
for c in cls:
  dic_keys, dictionary = preprocess_each_class(c, selected_features, depth_layer, state, num_neighbor)
  for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
  df = pd.DataFrame.from_dict(dictionary) 
  df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {c}.csv', index = False, header=True)

# All Samples
dic_keys, dictionary = preprocess_all_samples(selected_features, depth_layer, state, num_neighbor)
for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
df = pd.DataFrame.from_dict(dictionary) 
df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme ALL Samples.csv', index = False, header=True)

**Scenario 6: 2 layer before, 3 nearest neighbor**

---



In [ ]:
depth_layer = 2
num_neighbor = 3
state = '3D'
scen = 'Scenario 6'

# Each Sample
flag = ['Sample 4','Sample 5','Sample 6','Sample 8']
for f in flag:
    print(f)
    dic_keys, dictionary = preprocess_each_sample(f, selected_features, depth_layer, state, num_neighbor)
    for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
    df = pd.DataFrame.from_dict(dictionary) 
    df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {f}.csv', index = False, header=True)

# Each Class
cls = ['Class 1','Class 2']
for c in cls:
  dic_keys, dictionary = preprocess_each_class(c, selected_features, depth_layer, state, num_neighbor)
  for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
  df = pd.DataFrame.from_dict(dictionary) 
  df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {c}.csv', index = False, header=True)

# All Samples
dic_keys, dictionary = preprocess_all_samples(selected_features, depth_layer, state, num_neighbor)
for k in flatten(dic_keys):
    dictionary[k] = list(flatten(dictionary[k]))     
    
df = pd.DataFrame.from_dict(dictionary) 
df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme ALL Samples.csv', index = False, header=True)

**Scenario 7: 0 layer before, 2 nearest neighbor**

---



In [ ]:
depth_layer = 0
num_neighbor = 2
state = '2D'
scen = 'Scenario 7'

# Each Sample
flag = ['Sample 4','Sample 5','Sample 6','Sample 8']
for f in flag:
    print(f)
    dic_keys, dictionary = preprocess_each_sample(f, selected_features, depth_layer, state, num_neighbor)
    for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
    df = pd.DataFrame.from_dict(dictionary) 
    df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {f}.csv', index = False, header=True)

# Each Class
cls = ['Class 1','Class 2']
for c in cls:
  dic_keys, dictionary = preprocess_each_class(c, selected_features, depth_layer, state, num_neighbor)
  for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
  df = pd.DataFrame.from_dict(dictionary) 
  df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {c}.csv', index = False, header=True)

# All Samples
dic_keys, dictionary = preprocess_all_samples(selected_features, depth_layer, state, num_neighbor)
for k in flatten(dic_keys):
    dictionary[k] = list(flatten(dictionary[k]))     
    
df = pd.DataFrame.from_dict(dictionary) 
df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme ALL Samples.csv', index = False, header=True)

**Scenario 8: 0 layer before, 3 nearest neighbor**

---



In [ ]:
depth_layer = 0
num_neighbor = 3
state = '2D'
scen = 'Scenario 8'

# Each Sample
flag = ['Sample 4','Sample 5','Sample 6','Sample 8']
for f in flag:
    print(f)
    dic_keys, dictionary = preprocess_each_sample(f, selected_features, depth_layer, state, num_neighbor)

    for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
    df = pd.DataFrame.from_dict(dictionary) 
    df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {f}.csv', index = False, header=True)
    break

# Each Class
cls = ['Class 1','Class 2']
for c in cls:
  dic_keys, dictionary = preprocess_each_class(c, selected_features, depth_layer, state, num_neighbor)
  for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
  df = pd.DataFrame.from_dict(dictionary) 
  df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {c}.csv', index = False, header=True)


# All Samples
dic_keys, dictionary = preprocess_all_samples(selected_features, depth_layer, state, num_neighbor)
for k in flatten(dic_keys):
      dictionary[k] = list(flatten(dictionary[k]))     
    
df = pd.DataFrame.from_dict(dictionary) 
df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme ALL Samples.csv', index = False, header=True)

**Combine scenarios: 9, 10**

---



In [ ]:
def combine(f,nb,smp):
    dic_keys = []
    dic_list = []
    for s in state:
      if s == '3D':
        depth_layer = 2
        num_neighbor = nb
      elif s == '2D':
        depth_layer = 2
        num_neighbor = 2  
      # --------------------------------------
      if smp == 'Each sample':
        key, dictionary = preprocess_each_sample(f, selected_features, depth_layer, s, num_neighbor)
      if smp == 'class':
        key, dictionary = preprocess_each_class(f, selected_features, depth_layer, s, num_neighbor)
      if smp == 'All':
        key, dictionary = preprocess_all_samples(selected_features, depth_layer, s, num_neighbor)
      # ---------------------------------------
      dic_keys.append(key)
      dic_list.append(dictionary)

    dic1 = dic_list[0]
    dic2 = dic_list[1]

    for fea in selected_features:
      del dic2[fea] 
    dic1.update(dic2)

    for k in flatten(dic_keys):
      dic1[k] = list(flatten(dic1[k]))     
    
    df = pd.DataFrame.from_dict(dic1) 
    label_col = df.pop('Label')
    df.insert(len(df.columns), 'Label', label_col)

    return df

**Scenario 9: 2 layer before in each 1 NN, Current layer 2 nearest neighbor**

---



In [ ]:

state = ['3D','2D']
scen = 'Scenario 9'
nb = 1
# Each Sample
flag = ['Sample 4','Sample 5','Sample 6','Sample 8']
smp = 'Each sample'
for f in flag:
    print(f)
    df = combine(f,nb,smp)
    df.to_csv(f'/content/drive/MyDrive/DED Codes/Porosity prediction in DED/Neighborhood Schemes - Mutual Information Reg - 10 F/{scen}/Scheme {f}.csv', index = False, header=True)
    break


# # Each Class
# cls = ['Class 1','Class 2']
# for c in cls:
#   smp = 'class'
#   df = combine(c,nb,smp)
#   df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {c}.csv', index = False, header=True)

# # All Samples
# smp = 'All'
# df = combine(f,nb,smp)
# df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme ALL Samples.csv', index = False, header=True)

Sample 4
dic keys: ['pos_x', 'pos_y', 'pos_z', 'dist', 'sat', 'LongPeak', 'ShortPeak', 'Time', 'XPeak', 'TGrad', 'pos_x-1|N1', 'pos_y-1|N1', 'pos_z-1|N1', 'dist-1|N1', 'sat-1|N1', 'LongPeak-1|N1', 'ShortPeak-1|N1', 'Time-1|N1', 'XPeak-1|N1', 'TGrad-1|N1', 'Label-1|N1', 'pos_x-2|N1', 'pos_y-2|N1', 'pos_z-2|N1', 'dist-2|N1', 'sat-2|N1', 'LongPeak-2|N1', 'ShortPeak-2|N1', 'Time-2|N1', 'XPeak-2|N1', 'TGrad-2|N1', 'Label-2|N1', 'Label']
dic keys: ['pos_x', 'pos_y', 'pos_z', 'dist', 'sat', 'LongPeak', 'ShortPeak', 'Time', 'XPeak', 'TGrad', 'pos_x|N1', 'pos_y|N1', 'pos_z|N1', 'dist|N1', 'sat|N1', 'LongPeak|N1', 'ShortPeak|N1', 'Time|N1', 'XPeak|N1', 'TGrad|N1', 'Label|N1', 'pos_x|N2', 'pos_y|N2', 'pos_z|N2', 'dist|N2', 'sat|N2', 'LongPeak|N2', 'ShortPeak|N2', 'Time|N2', 'XPeak|N2', 'TGrad|N2', 'Label|N2', 'Label']


**Scenario 10: 2 layer before in each 2 NN, Current layer 2 nearest neighbor**

---



In [ ]:


state = ['3D','2D']
scen = 'Scenario 10'
nb = 2
# Each Sample
flag = ['Sample 4','Sample 5','Sample 6','Sample 8']
smp = 'Each sample'
for f in flag:
    print(f)
    df = combine(f,nb,smp)
    df.to_csv(f'/content/drive/MyDrive/DED Codes/Porosity prediction in DED/Neighborhood Schemes - Mutual Information Reg - 10 F/{scen}/Scheme {f}.csv', index = False, header=True)
    break


# Each Class
# cls = ['Class 1','Class 2']
# for c in cls:
#   smp = 'class'
#   df = combine(c,nb,smp)
#   df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme {c}.csv', index = False, header=True)

# # All Samples
# smp = 'All'
# df = combine(f,nb,smp)
# df.to_csv(f'/content/drive/MyDrive/DED Codes/Dataset Pointwise/{scen}/Scheme ALL Samples.csv', index = False, header=True)

Sample 4
dic keys: ['pos_x', 'pos_y', 'pos_z', 'dist', 'sat', 'LongPeak', 'ShortPeak', 'Time', 'XPeak', 'TGrad', 'pos_x-1|N1', 'pos_y-1|N1', 'pos_z-1|N1', 'dist-1|N1', 'sat-1|N1', 'LongPeak-1|N1', 'ShortPeak-1|N1', 'Time-1|N1', 'XPeak-1|N1', 'TGrad-1|N1', 'Label-1|N1', 'pos_x-1|N2', 'pos_y-1|N2', 'pos_z-1|N2', 'dist-1|N2', 'sat-1|N2', 'LongPeak-1|N2', 'ShortPeak-1|N2', 'Time-1|N2', 'XPeak-1|N2', 'TGrad-1|N2', 'Label-1|N2', 'pos_x-2|N1', 'pos_y-2|N1', 'pos_z-2|N1', 'dist-2|N1', 'sat-2|N1', 'LongPeak-2|N1', 'ShortPeak-2|N1', 'Time-2|N1', 'XPeak-2|N1', 'TGrad-2|N1', 'Label-2|N1', 'pos_x-2|N2', 'pos_y-2|N2', 'pos_z-2|N2', 'dist-2|N2', 'sat-2|N2', 'LongPeak-2|N2', 'ShortPeak-2|N2', 'Time-2|N2', 'XPeak-2|N2', 'TGrad-2|N2', 'Label-2|N2', 'Label']
dic keys: ['pos_x', 'pos_y', 'pos_z', 'dist', 'sat', 'LongPeak', 'ShortPeak', 'Time', 'XPeak', 'TGrad', 'pos_x|N1', 'pos_y|N1', 'pos_z|N1', 'dist|N1', 'sat|N1', 'LongPeak|N1', 'ShortPeak|N1', 'Time|N1', 'XPeak|N1', 'TGrad|N1', 'Label|N1', 'pos_x|N2',